In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:

width, height = 280, 560

def detect_blue_ball_red_table(transformed_img):
    """
    Detects the BLUE ball on RED pool table
    """
    blurred = cv2.GaussianBlur(transformed_img, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Blue color range
    lower_blue = np.array([90, 90, 50])
    upper_blue = np.array([130, 255, 255])

    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    kernel = np.ones((7, 7), np.uint8)
    mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel)
    mask_blue = cv2.GaussianBlur(mask_blue, (5, 5), 0)

    contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    blue_ball = None
    max_score = 0

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 30 < area < 900:  
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            score = circularity * area
            if score > max_score:
                max_score = score
                blue_ball = cnt

    return blue_ball, mask_blue

def detect_black_ball_red_table(transformed_img):
    """
    Detects BLACK ball on red table using darkness
    """
    blurred = cv2.GaussianBlur(transformed_img, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # black one
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 70])
    
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    
    kernel = np.ones((7, 7), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_OPEN, kernel)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)
    
    contours, _ = cv2.findContours(mask_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    black_ball = None
    max_score = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 30 < area < 900:
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            score = circularity * area
            if score > max_score:
                max_score = score
                black_ball = cnt
    
    return black_ball, mask_black

def detect_white_ball_red_table(transformed_img):
    """
    Detects WHITE ball on red table using brightness
    """
    blurred = cv2.GaussianBlur(transformed_img, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # white one
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 50, 255])
    
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    
    kernel = np.ones((5, 5), np.uint8)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)
    
    contours, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    white_ball = None
    max_score = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 30 < area < 900:
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            score = circularity * area
            if score > max_score:
                max_score = score
                white_ball = cnt
    
    return white_ball, mask_white

def track_ball_two_outputs(video_path, ball_type='white', 
                           output_tracked='tracked_with_circle.mp4',
                           output_ballonly='ball_only_blackbg.mp4'):

    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return
    

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    orig_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    orig_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Input video: {video_path}")
    print(f"Size: {orig_w}x{orig_h}, FPS: {fps:.2f}, Frames: {total_frames}")
    
 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_tracked = cv2.VideoWriter(output_tracked, fourcc, fps, (orig_w, orig_h))
    out_ballonly = cv2.VideoWriter(output_ballonly, fourcc, fps, (orig_w, orig_h))
 
    pts1 = np.float32([[255, 60], [590, 60], [160, 380], [690, 380]])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    frame_num = 0
    trajectory = []  
    
    print(f"Tracking {ball_type.upper()} ball...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
      
        transformed = cv2.warpPerspective(frame, matrix, (width, height))
        
       
        if ball_type == "blue":
            ball_contour, ball_mask = detect_blue_ball_red_table(transformed)
        elif ball_type == "black":
            ball_contour, ball_mask = detect_black_ball_red_table(transformed)
        else:  
            ball_contour, ball_mask = detect_white_ball_red_table(transformed)
        
   
        tracked_frame = frame.copy()
        
       
        ball_only_frame = np.zeros_like(frame)
        
        if ball_contour is not None:
          
            M = cv2.moments(ball_contour)
            if M['m00'] != 0:
                cX_trans = int(M['m10'] / M['m00'])
                cY_trans = int(M['m01'] / M['m00'])
                
                (x, y), radius = cv2.minEnclosingCircle(ball_contour)
                radius = int(radius)
                

                pts_transformed = np.array([[[cX_trans, cY_trans]]], dtype=np.float32)
                inv_matrix = cv2.getPerspectiveTransform(pts2, pts1)
                pts_original = cv2.perspectiveTransform(pts_transformed, inv_matrix)
                
                cX_orig = int(pts_original[0][0][0])
                cY_orig = int(pts_original[0][0][1])
                
                radius_orig = int(radius * (orig_w / width))
                
               
                trajectory.append((cX_orig, cY_orig))
                
                cv2.circle(tracked_frame, (cX_orig, cY_orig), radius_orig, (0, 255, 0), 2)
                cv2.circle(tracked_frame, (cX_orig, cY_orig), 2, (0, 0, 255), -1)
               
                for i in range(1, len(trajectory)):
                    cv2.line(tracked_frame, trajectory[i-1], trajectory[i], (0, 255, 255), 2)
                
           
                mask_orig = np.zeros((orig_h, orig_w), dtype=np.uint8)
                cv2.circle(mask_orig, (cX_orig, cY_orig), max(1, radius_orig - 2), 255, -1)
                
         
                for c in range(3):
                    ball_only_frame[:, :, c] = cv2.bitwise_and(frame[:, :, c], frame[:, :, c], mask=mask_orig)

        out_tracked.write(tracked_frame)
        out_ballonly.write(ball_only_frame)

        if frame_num % 30 == 0:
            print(f"Processing frame {frame_num}/{total_frames}")
        
        frame_num += 1

        display_scale = 0.5
        tracked_display = cv2.resize(tracked_frame, None, fx=display_scale, fy=display_scale)
        ballonly_display = cv2.resize(ball_only_frame, None, fx=display_scale, fy=display_scale)
        
        cv2.imshow('Tracked Video', tracked_display)
        cv2.imshow('Ball Only', ballonly_display)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out_tracked.release()
    out_ballonly.release()
    cv2.destroyAllWindows()
    
    print(f"\n  DONE! Two videos created:")
    print(f"1. Tracked video: {output_tracked}")
    print(f"2. Ball only video: {output_ballonly}")
    print(f"Frames processed: {frame_num}")

if __name__ == "__main__":
    import os
    
    VIDEO_IN = "gti./pooll6.mp4" #put your video path CTRL+ALT+C
    
    BALL_COLOR = "white"

    VIDEO_OUT_TRACKED = f"tracked_{BALL_COLOR}_ball.mp4"
    VIDEO_OUT_BALLONLY = f"{BALL_COLOR}_ball_only_blackbg.mp4"
    if not os.path.exists(VIDEO_IN):
        print(f"ERROR: Video file '{VIDEO_IN}' not found!")
        print(f"Current directory: {os.getcwd()}")
    else:
        track_ball_two_outputs(VIDEO_IN, 
                              ball_type=BALL_COLOR,
                              output_tracked=VIDEO_OUT_TRACKED,
                              output_ballonly=VIDEO_OUT_BALLONLY)

Input video: ./pooll6.mp4
Size: 640x360, FPS: 30.00, Frames: 4399
Tracking WHITE ball...
Processing frame 0/4399
Processing frame 30/4399
Processing frame 60/4399
Processing frame 90/4399
Processing frame 120/4399
Processing frame 150/4399
Processing frame 180/4399
Processing frame 210/4399
Processing frame 240/4399
Processing frame 270/4399
Processing frame 300/4399
Processing frame 330/4399
Processing frame 360/4399
Processing frame 390/4399
Processing frame 420/4399
Processing frame 450/4399
Processing frame 480/4399
Processing frame 510/4399
Processing frame 540/4399
Processing frame 570/4399
Processing frame 600/4399
Processing frame 630/4399
Processing frame 660/4399
Processing frame 690/4399
Processing frame 720/4399
Processing frame 750/4399
Processing frame 780/4399
Processing frame 810/4399
Processing frame 840/4399
Processing frame 870/4399
Processing frame 900/4399
Processing frame 930/4399
Processing frame 960/4399
Processing frame 990/4399
Processing frame 1020/4399
Proce